In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
train_data = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/rusentitweet_train.csv')
test_data = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/rusentitweet_test.csv')

true_labels = ['positive', 'negative']
filtered_train_data = train_data[train_data['label'].isin(true_labels)]
filtered_test_data = test_data[test_data['label'].isin(true_labels)]

filtered_train_data.head(6)

,text,label,id
0,Помойму я вкрашилась в Чимина🤧 https://t.co/t2...,positive,1282311169534038016
5,@buybread_ я не с порядке!!!!,negative,1335130757044563971
10,@ange1flyhigh В следующий раз буду до победног...,positive,1215370396465291267
15,@LimitaVIP Удивительный гiмн...\r\nУдивительно...,negative,1253799540848762887
17,я срала на эту биологию,negative,1339418979887173632
18,"Помимо Алины, у меня ещё радость...\r\n\r\nГЕМ...",positive,1332289032160751616


In [4]:
import re
# Очистка текста
def cleaner(tweet):
  #Удаление ссылок на аккаунты
  tweet = re.sub(r'@[\w]+', '', tweet)

  #Удаление обычных ссылок
  tweet = re.sub(r'https?://\S+', '', tweet)

  #Удаление спецсимволов "^", "&", "*", "|", "/"
  tweet = re.sub(r'[\^&*/|]', '', tweet)

  #Удаление хэштегов
  tweet = re.sub(r'#\w+', '', tweet)

  return tweet

cleaned_train_data = filtered_train_data.copy()
cleaned_test_data = filtered_test_data.copy()

cleaned_train_data['text'] = cleaned_train_data['text'].apply(cleaner)
cleaned_test_data['text'] = cleaned_test_data['text'].apply(cleaner)

cleaned_train_data.head(6)

,text,label,id
0,Помойму я вкрашилась в Чимина🤧,positive,1282311169534038016
5,я не с порядке!!!!,negative,1335130757044563971
10,В следующий раз буду до победного ее закрыват...,positive,1215370396465291267
15,"Удивительный гiмн...\r\nУдивительно, что пока...",negative,1253799540848762887
17,я срала на эту биологию,negative,1339418979887173632
18,"Помимо Алины, у меня ещё радость...\r\n\r\nГЕМ...",positive,1332289032160751616


In [5]:
import nltk
from nltk.stem.snowball import SnowballStemmer
#Осуществим стемминг подготовленного набора данных
stemmer = SnowballStemmer('russian')
def stemming(tweet):
  words_arr = tweet.split()
  words_arr = list(map(lambda w: stemmer.stem(w), words_arr))
  return ' '.join(words_arr)

# Применение стемминга
stemmed_train_data = cleaned_train_data.copy()
stemmed_test_data = cleaned_test_data.copy()
stemmed_train_data['text'] = stemmed_train_data['text'].apply(stemming)
stemmed_test_data['text'] = stemmed_test_data['text'].apply(stemming)

stemmed_train_data.head(6)

,text,label,id
0,помойм я вкраш в чимина🤧,positive,1282311169534038016
5,я не с порядке!!!!,negative,1335130757044563971
10,в след раз буд до победн е закрыва пожела удачи!,positive,1215370396465291267
15,"удивительн гимн... удивительно, что пок ещ не ...",negative,1253799540848762887
17,я срал на эт биолог,negative,1339418979887173632
18,"помим алины, у мен ещ радость... гемоглобин повыс",positive,1332289032160751616


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
#Преобразование в мешок слов
vectorizer = CountVectorizer()
stemmed_train_data_counts = vectorizer.fit_transform(stemmed_train_data['text'])
stemmed_test_data_counts = vectorizer.transform(stemmed_test_data['text'])
print(stemmed_train_data_counts)

  (0, 6181)	1
  (0, 1092)	1
  (0, 9358)	1
  (1, 4565)	1
  (1, 6290)	1
  (2, 7631)	1
  (2, 6962)	1
  (2, 819)	1
  (2, 2021)	1
  (2, 5836)	1
  (2, 2600)	1
  (2, 6023)	1
  (2, 8675)	1
  (3, 4565)	1
  (3, 8681)	1
  (3, 1552)	1
  (3, 8682)	1
  (3, 9394)	1
  (3, 6067)	1
  (3, 2353)	1
  (3, 7425)	1
  (3, 8731)	1
  (4, 7974)	1
  (4, 4330)	1
  (4, 9615)	1
  :	:
  (4566, 4187)	1
  (4566, 8884)	1
  (4566, 872)	1
  (4566, 9134)	1
  (4566, 2038)	1
  (4567, 4565)	1
  (4567, 8704)	1
  (4567, 4782)	1
  (4567, 4789)	1
  (4567, 1311)	1
  (4567, 8858)	1
  (4567, 2548)	1
  (4567, 1245)	1
  (4568, 8250)	1
  (4568, 4788)	1
  (4568, 1268)	2
  (4568, 8303)	1
  (4568, 4306)	1
  (4568, 9166)	1
  (4568, 446)	1
  (4568, 945)	1
  (4568, 6359)	1
  (4568, 2871)	1
  (4568, 8827)	1
  (4568, 4551)	1


In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
#TF-IDF
tfidf = TfidfTransformer()
#Обучение на тренировочных данных
d_train_tfidf = tfidf.fit_transform(stemmed_train_data_counts)
#Преобразование текста в Tf-Idf матрицу на тестовых данных
d_test_tfidf = tfidf.transform(stemmed_test_data_counts)

print(d_test_tfidf)

  (0, 1612)	0.4667312383075685
  (0, 4033)	0.6315223785221268
  (0, 8197)	0.5675729134204611
  (0, 9615)	0.24739123785018977
  (1, 1268)	0.14442049052410402
  (1, 1521)	0.23567246721206728
  (1, 2206)	0.22932206751337117
  (1, 2353)	0.17646899685903608
  (1, 2679)	0.35054833315400247
  (1, 4330)	0.13091600877901785
  (1, 4369)	0.31377254568920676
  (1, 4565)	0.10526153044250353
  (1, 7281)	0.2696791912233885
  (1, 7510)	0.31377254568920676
  (1, 7870)	0.35054833315400247
  (1, 8250)	0.1278333851301533
  (1, 8295)	0.24462844540960532
  (1, 9381)	0.30645497868818417
  (1, 9615)	0.12005002372274143
  (1, 9680)	0.33427478795544185
  (2, 748)	0.51574210567308
  (2, 1650)	0.7599572533044352
  (2, 4330)	0.30827944566869453
  (2, 4565)	0.2478685880947374
  (3, 3115)	0.26469370217941035
  :	:
  (1137, 7523)	0.2793444488322354
  (1137, 8884)	0.2775797266085705
  (1137, 9166)	0.22757486424830672
  (1138, 826)	0.3066832490241846
  (1138, 1756)	0.1908971921164272
  (1138, 2459)	0.18174132749426333


In [8]:
#Обучаем модели логистической регрессии и случайного леса на обучающей выборке, применяем их к тестовым данным
label_maps = {"positive":1, "negative":0}

#Сопоставление меток с числами
D_train_encoded = stemmed_train_data['label'].map(label_maps)
D_test_encoded = stemmed_test_data['label'].map(label_maps)

from sklearn.linear_model import LogisticRegression
#Логистическая регрессия
log_reg = LogisticRegression()
log_reg.fit(d_train_tfidf, D_train_encoded)

from sklearn.ensemble import RandomForestClassifier
#Случайный лес
rand_forest = RandomForestClassifier()
rand_forest.fit(d_train_tfidf, D_train_encoded)

#Предсказания
predict_lin_reg_train = log_reg.predict(d_train_tfidf)
predict_rand_for_train = rand_forest.predict(d_train_tfidf)

predict_lin_reg_test = log_reg.predict(d_test_tfidf)
predict_rand_for_test = rand_forest.predict(d_test_tfidf)

from sklearn.metrics import classification_report
#выводим полученные результаты по показателям
print("Обучающие данные\n")
print("Логистическая регрессия\n", classification_report(D_train_encoded,predict_lin_reg_train))
print("Случайный лес\n", classification_report(D_train_encoded,predict_rand_for_train))

print("Тестовые данные\n")
print("Логистическая регрессия\n", classification_report(D_test_encoded,predict_lin_reg_test))
print("Случайный лес\n", classification_report(D_test_encoded,predict_rand_for_test))

Обучающие данные

Логистическая регрессия
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      2638
           1       0.96      0.81      0.88      1931

    accuracy                           0.91      4569
   macro avg       0.92      0.90      0.90      4569
weighted avg       0.91      0.91      0.91      4569

Случайный лес
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2638
           1       1.00      1.00      1.00      1931

    accuracy                           1.00      4569
   macro avg       1.00      1.00      1.00      4569
weighted avg       1.00      1.00      1.00      4569

Тестовые данные

Логистическая регрессия
               precision    recall  f1-score   support

           0       0.74      0.92      0.82       660
           1       0.83      0.56      0.67       483

    accuracy                           0.76      1143
   macro avg       0.78     

In [9]:
#Выявляем наиболее важные признаки(слова)
n = 15

log_reg_coef = log_reg.coef_[0]
log_reg_ind = abs(log_reg_coef).argsort()[::-1]
log_reg_names = vectorizer.get_feature_names_out()
log_reg_top_names = [log_reg_names[index] for index in log_reg_ind[:n]]
print("Топ 25 наиболее значимых слов для логистической регрессии:\n", log_reg_top_names)

rand_forest_coef = rand_forest.feature_importances_
rand_forest_ind = abs(rand_forest_coef).argsort()[::-1]
rand_forest_names = vectorizer.get_feature_names_out()
rand_forest_top_names = [rand_forest_names[index] for index in rand_forest_ind[:n]]
print("Топ 25 наиболее значимых слов для случайного леса:\n", rand_forest_top_names)


Топ 25 наиболее значимых слов для логистической регрессии:
 ['не', 'любл', 'хорош', 'пиздец', 'блят', 'красив', 'нрав', 'нет', 'мил', 'классн', 'нах', 'крут', 'рад', 'хорошо', 'прекрасн']
Топ 25 наиболее значимых слов для случайного леса:
 ['не', 'так', 'любл', 'хорош', 'эт', 'что', 'нет', 'как', 'пиздец', 'красив', 'все', 'мо', 'блят', 'ты', 'на']


In [10]:
from pymystem3 import Mystem
#Лемматизация
m = Mystem()
def lemmatization(tweet):
  lem = m.lemmatize(tweet)
  lemmas = []
  for l in lem:
    if l.isalnum():
      lemmas.append(l)
  lem_tweet = ' '.join(lemmas)
  return lem_tweet

# Применение лемматизации
lem_train_data = cleaned_train_data.copy()
lem_test_data = cleaned_test_data.copy()
lem_train_data['text'] = lem_train_data['text'].apply(lemmatization)
lem_test_data['text'] = lem_test_data['text'].apply(lemmatization)

lem_train_data.head(50)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


,text,label,id
0,помойма я вкрашиться в чимин,positive,1282311169534038016
5,я не с порядок,negative,1335130757044563971
10,в следующий раз быть до победный она закрывать...,positive,1215370396465291267
15,удивительный гiмн удивительно что пока еще не ...,negative,1253799540848762887
17,я срать на этот биология,negative,1339418979887173632
18,помимо алина у я еще радость гемоглобин повыша...,positive,1332289032160751616
20,пиздец че вообще быть чувствовать как будто я ...,negative,1286564487374544897
22,зачем я спичка чтоб зажигать огонь в твой глаз,positive,1262165906450452481
25,это на самый дело очень круто,positive,1334947965048414211
27,хотеть сказать приятный но получаться идти нах...,negative,1220748047845601280


In [11]:
#Всё то же самое, но с лемматизацией

#Преобразование в мешок слов
vectorizer = CountVectorizer()
lem_train_data_counts = vectorizer.fit_transform(lem_train_data['text'])
lem_test_data_counts = vectorizer.transform(lem_test_data['text'])
print(lem_train_data_counts)

#################

#TF-IDF
tfidf = TfidfTransformer()
#Обучение на тренировочных данных
d_train_tfidf = tfidf.fit_transform(lem_train_data_counts)
#Преобразование текста в Tf-Idf матрицу на тестовых данных
d_test_tfidf = tfidf.transform(lem_test_data_counts)

print(d_test_tfidf)

#################

#Обучаем модели логистической регрессии и случайного леса на обучающей выборке, применяем их к тестовым данным
label_maps = {"positive":1, "negative":0}

#Сопоставление меток с числами
D_train_encoded = lem_train_data['label'].map(label_maps)
D_test_encoded = lem_test_data['label'].map(label_maps)

#Логистическая регрессия
log_reg = LogisticRegression()
log_reg.fit(d_train_tfidf, D_train_encoded)

#Случайный лес
rand_forest = RandomForestClassifier()
rand_forest.fit(d_train_tfidf, D_train_encoded)

#Предсказания
predict_lin_reg_train = log_reg.predict(d_train_tfidf)
predict_rand_for_train = rand_forest.predict(d_train_tfidf)

predict_lin_reg_test = log_reg.predict(d_test_tfidf)
predict_rand_for_test = rand_forest.predict(d_test_tfidf)

from sklearn.metrics import classification_report
#выводим полученные результаты по показателям
print("Обучающие данные\n")
print("Линейная регрессия\n", classification_report(D_train_encoded,predict_lin_reg_train))
print("Случайный лес\n", classification_report(D_train_encoded,predict_rand_for_train))

print("Тестовые данные\n")
print("Линейная регрессия\n", classification_report(D_test_encoded,predict_lin_reg_test))
print("Случайный лес\n", classification_report(D_test_encoded,predict_rand_for_test))

#######################

#Выявляем наиболее важные признаки(слова)
n = 15

log_reg_coef = log_reg.coef_[0]
log_reg_ind = abs(log_reg_coef).argsort()[::-1]
log_reg_names = vectorizer.get_feature_names_out()
log_reg_top_names = [log_reg_names[index] for index in log_reg_ind[:n]]
print("Топ 25 наиболее значимых слов для логистической регрессии:\n", log_reg_top_names)
log_reg_sort_coef = sorted(log_reg.coef_[0])
print(log_reg_sort_coef)

rand_forest_coef = rand_forest.feature_importances_
rand_forest_ind = abs(rand_forest_coef).argsort()[::-1]
rand_forest_names = vectorizer.get_feature_names_out()
rand_forest_top_names = [rand_forest_names[index] for index in rand_forest_ind[:n]]
print("Топ 25 наиболее значимых слов для случайного леса:\n", rand_forest_top_names)

  (0, 4979)	1
  (0, 983)	1
  (0, 7446)	1
  (1, 3702)	1
  (1, 5046)	1
  (2, 6089)	1
  (2, 5555)	1
  (2, 801)	1
  (2, 1713)	1
  (2, 4733)	1
  (2, 4158)	1
  (2, 2135)	1
  (2, 4870)	1
  (2, 6896)	1
  (3, 3702)	1
  (3, 6903)	1
  (3, 1279)	1
  (3, 6902)	1
  (3, 7473)	1
  (3, 4897)	1
  (3, 1968)	1
  (3, 5932)	1
  (3, 6934)	1
  (4, 6331)	1
  (4, 3531)	1
  :	:
  (4566, 3469)	1
  (4566, 6689)	1
  (4566, 7038)	1
  (4566, 1722)	1
  (4566, 7261)	1
  (4567, 3702)	1
  (4567, 6927)	1
  (4567, 3898)	1
  (4567, 3909)	1
  (4567, 1154)	1
  (4567, 7016)	1
  (4567, 2098)	1
  (4567, 1115)	1
  (4568, 6546)	1
  (4568, 3903)	1
  (4568, 1130)	2
  (4568, 7287)	1
  (4568, 6591)	1
  (4568, 3512)	1
  (4568, 3695)	1
  (4568, 863)	1
  (4568, 5094)	1
  (4568, 429)	1
  (4568, 2346)	1
  (4568, 6993)	1
  (0, 1380)	0.43636302824609546
  (0, 3294)	0.6451344123364038
  (0, 6504)	0.562262831341717
  (0, 7664)	0.2779377737747353
  (1, 907)	0.19969790199821386
  (1, 1311)	0.29074494658078914
  (1, 1854)	0.2597768795463781
  (1,

In [12]:
from sklearn.metrics import roc_auc_score
print('Logistic Regression ROC-AUC:', roc_auc_score(D_train_encoded, predict_lin_reg_train))
print('Random Forest ROC-AUC:', roc_auc_score(D_train_encoded, predict_rand_for_train))

Logistic Regression ROC-AUC: 0.8948110887012076
Random Forest ROC-AUC: 0.9986038416341806
